# BIG DATA PROJECT : Genomics drug response prediction

## Predict Drug Response on basis of Gene Expression
 Aiding the medical Industry in the drug testing scenarios
 
 Team 5 :
 Members - 
 1. Mrudula Murali
 2. Sudeep Lunawat
 3. Sagar Kulkarni 
 4. Rohan Sunki

# QUESTION WE NEED TO ANSWER WITH THIS PROJECT(ANALYSIS)

BASED UPON : Genomics Benchmarking using Big Data

[REFERENCES]

GenBase: a complex analytics genomics benchmark

https://github.com/mitdbg/genbase

Big Data Science & Analytics: A Hands-On Approach

Q1) Why we need BIG DATA for this Project :

Ans) In order to understand Drug Responses on different patients , It indespensible that we segregate then on basis of their Genomics. Genomics is quickly becoming a major source of big data due to advances in sequencing technology. It is now possible to sequence over 2000 people per day at a sequencing facility. At 3 GB/genome, such a facility produces 6TB of data per day.

Genomics data can be extremely large scale. With 10^5 gene
expressions per sample and up to 10^10 samples (multiple samples
can be taken from one patient), we are looking at a problem
that can scale to 10^5 nodes of a large cluster.

Q1.1) Why gene expressions?
Ans) There are facilities that analyze genes on basis of chromoses, dna code etc to formulate a single metric called as gene expression. This expression defines the behaviour of the genes and can be genralized on genes rather than individual patients.


Q2) How to Predict the Drug response for Different Genes and their Expressions?

ANS) Seeing that we need to predict drug response on gene-sets, Regression Analysis seems to be the way to go. We have used Linear Regression and Decision tree regressor for the predictions.

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.mllib.regression import LinearRegressionModel
from pyspark.sql import types 

In [4]:
sqlContext = SQLContext(sc)

In [5]:
genes = sc.textFile('/FileStore/tables/GeneMetaData_1000_1000-06784.txt')
header = genes.first() #extract header
genes = genes.filter(lambda x:x !=header)

In [6]:
gparts = genes.map(lambda l: l.split(", "))
geneframe = gparts.map(lambda p: Row(geneid=int(p[0]),
target=int(p[1]), position = int(p[2]),
length=int(p[3]), function=int(p[4])))

In [7]:
schemaGene = sqlContext.createDataFrame(geneframe)
schemaGene.registerTempTable("genes")

In [8]:
patients = sc.textFile('/FileStore/tables/PatientMetaData_1000_1000-4f01f.txt')
header = patients.first() #extract header
patients = patients.filter(lambda x:x !=header)

In [9]:
pparts = patients.map(lambda l: l.split(", "))
patientsframe = pparts.map(lambda p: Row(patientid=int(p[0]),
age=int(p[1]), gender=int(p[2]),
zipcode=int(p[3]), disease=int(p[4]),
drugResponse = float(p[5])))

In [10]:
schemaPatients = sqlContext.createDataFrame(patientsframe)
schemaPatients.registerTempTable("patients")

In [11]:
geo = sc.textFile('/FileStore/tables/GEO_1000_1000-f11e4.txt') #extracting Gene Expression Omnibus
header = geo.first() #extract header
geo = geo.filter(lambda x:x !=header)

--------------------------------------------------------------------------- 
 Exception Traceback (most recent call last)
 <command-2940114943719727> in <module> 
 ----> 1 display ( geo ) 

 /local_disk0/tmp/1588903553840-0/PythonShell.py in display (self, input, *args, **kwargs) 
 1109 self . displayHTML ( input . _repr_html_ ( ) ) 
 1110 else : 
 -> 1111 raise Exception ( genericErrorMsg ) 
 1112 
 1113 def displayHTML ( self , html ) : 

 Exception : Cannot call display(<class 'pyspark.rdd.PipelinedRDD'>)
 Call help(display) for more info.

In [13]:
geoparts = geo.map(lambda l: l.split(", "))
geoframe = geoparts.map(lambda p: Row(geneid=str(p[0]),
patientid=int(p[1]), exValue = float(p[2])))

In [14]:
schemaGEO = sqlContext.createDataFrame(geoframe)
schemaGEO.registerTempTable("geo")

In [15]:
display(schemaGEO)

exValue,geneid,patientid
874417.23,0,0
226331.06,0,1
47877.24,0,2
1579900.21,0,3
-10849.22,0,4
1674022.97,0,5
1894835.87,0,6
494663.29,0,7
1075574.37,0,8
407972.65,0,9


In [16]:
# Select genes with a particular set of functions and join the gene metadata with patient
g = sqlContext.sql("SELECT p.patientid, p.disease,e.geneid, e.exValue, p.drugResponse FROM genes AS g, patients AS p, geo AS e WHERE g.function < 300 AND g.geneid = e.geneid AND p.patientid = e.patientid")

In [17]:
g.registerTempTable("responses")
g2=g.groupBy('patientid').pivot('geneid').sum('exValue')
g2=g2.withColumnRenamed('patientid','geo_patientid')
g2.registerTempTable("gen")
g3 = sqlContext.sql("SELECT patientid, disease, drugResponse FROM patients")
g3.registerTempTable("gen3")
#g4 = sqlContext.sql("SELECT gen3.patientid,disease,drugResponse,0,2,5,6,8 FROM gen3 join gen on gen3.patientid= gen.patientid")
g4 = sqlContext.sql("SELECT gen3.patientid,gen3.disease,gen3.drugResponse,gen.* FROM gen3 join gen on gen3.patientid= gen.geo_patientid")

In [19]:
display(g4)

patientid,disease,drugResponse,geo_patientid,100,106,11,116,118,120,121,123,126,136,14,140,141,144,145,149,151,152,154,16,161,163,164,169,172,174,175,182,190,194,197,198,199,201,206,207,21,210,211,215,218,22,222,224,23,230,231,233,238,240,243,252,255,256,259,261,263,265,266,267,268,269,27,271,273,28,285,286,289,29,293,296,297,298,306,308,313,315,316,319,320,321,329,332,339,341,344,346,347,350,351,354,355,356,357,361,363,366,371,372,375,378,380,382,383,386,392,395,4,404,407,409,415,416,417,423,427,43,432,435,437,438,439,444,445,446,45,454,455,459,46,47,470,472,48,502,505,510,511,520,521,524,529,53,533,539,542,543,548,55,551,552,558,56,562,565,566,567,573,576,577,58,583,584,585,586,589,591,592,599,605,608,609,611,617,619,629,630,631,632,633,634,637,638,639,647,648,649,651,652,654,656,657,66,661,663,664,667,669,67,673,675,677,678,68,681,685,687,688,692,694,699,704,706,710,713,715,716,723,729,730,731,732,734,74,741,749,751,752,756,757,762,763,764,776,777,778,779,780,79,792,793,794,799,8,801,804,81,810,818,821,823,825,828,829,830,834,84,846,853,855,856,857,859,86,862,870,872,879,88,882,886,89,892,894,895,897,901,904,908,909,915,919,920,921,922,926,93,931,935,938,940,942,944,948,95,958,963,966,972,975,976,978,98,980,983,984,985,986,988,989,990,991,994,999
26,7,54.98,26,1611538.32,1612399.7,1710029.77,341599.12,309506.9,-82512.74,1467278.81,777050.32,655762.91,79762.18,355277.91,1148079.15,618890.68,1486815.26,838700.07,1311049.24,584543.15,1799447.45,1720323.78,524822.18,708293.49,1669119.97,1205917.92,360160.02,1821465.82,682553.11,127477.39,1356039.67,1041200.26,280445.75,1117792.55,1136739.5,784024.29,1383117.68,1737136.03,408537.27,172897.78,-143086.21,789665.04,1526604.36,1097451.4,484639.52,497708.72,982601.03,466998.79,418310.3,1935746.09,-109533.06,1652290.49,291798.46,1529142.65,1754426.44,1599571.05,1945774.39,383178.23,1614331.28,339866.43,1639541.97,593154.51,251837.28,568685.48,1211479.36,1148202.07,1998289.66,-60827.68,586148.06,218217.29,407783.64,332523.05,1435381.18,316696.59,1710203.85,1717341.05,1642077.58,55460.28,1679321.14,19790.03,1429140.11,-10925.6,487569.88,956500.09,1796621.82,955231.45,1371888.8,1916058.17,1649588.05,1096964.05,1524837.09,-26803.56,1009829.56,723349.02,205728.13,233727.22,1020567.65,1770108.0,534296.71,337417.19,1829593.85,1820298.67,948635.96,116495.7,1088304.89,-156209.15,905801.61,1605803.92,184481.22,417849.65,810195.35,761579.66,1452538.14,-94094.14,487517.9,-146583.1,540879.27,1410890.88,437583.32,888281.28,330427.8,1692856.05,1055360.6,751640.13,1432541.41,993309.4,985148.28,867809.66,1212150.5,779302.11,1563307.7,857414.94,1708939.38,1261727.7,440585.56,1883498.17,239793.55,1942570.65,1094527.96,1533287.92,1077920.66,1605444.65,1259907.82,211615.6,112966.48,297430.43,423594.51,1710513.28,1059232.69,1794146.87,1641788.36,1390884.19,176338.61,1157687.25,252669.34,1555605.89,1499195.5,32926.86,1914081.68,1427713.9,1381055.28,135112.41,1035908.63,767998.85,1205663.87,1482492.75,1356309.69,1386572.35,548038.92,1724104.15,457742.48,1970366.69,943257.87,1978301.82,118049.38,829348.38,836103.89,1184255.26,1092833.62,1990221.55,109403.25,1853336.57,647755.96,323561.21,1828019.62,1615020.55,877568.28,846288.35,1215430.5,557390.79,442775.93,683530.43,1763414.16,409446.04,1633848.23,1411333.93,86790.76,1781181.67,400592.03,506580.51,965755.45,218849.83,903669.78,136869.47,1219206.2,1223665.93,-51595.94,1618402.78,1758835.1,-116390.3,389150.24,1616259.63,371266.45,1021883.27,647874.14,1428742.61,480709.75,1532372.81,1260801.84,254447.76,691192.11,373606.59,996052.26,347739.3,1213636.14,907444.88,1239319.01,1361827.32,59892.75,781492.69,571362.53,1177322.42,510662.51,1152770.88,1194467.27,1003580.93,198877.02,1584686.24,1757123.53,-185984.09,543000.24,1097234.03,815990.9,1549957.13,261286.12,293689.32,1906039.12,940004.37,737156.69,314861.97,1141112.3,510383.3,1675619.69,1549591.77,343471.94,1412049.31,-100657.67,923397.95,403173.74,-112229.47,1025994.71,1567987.32,987102.52,815679.24,532539.93

# Exploratory Data Analysis

In [21]:
### Data Cleaning

# Checking for null values
from pyspark.sql.functions import isnan, when, count, col

g4.select([count(when(isnan(c), c)).alias(c) for c in g4.columns]).show()

## From the output we can see that there are no null values in our data

+---------+-------+------------+-------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
patientid|disease|drugResponse|geo_patientid|100|106| 11|116|118|120|121|123|126|136| 14|140|141|144|145|149|151|152|154| 16|161|163|164|169|172|174|175|182|190|194|197|198|199|201|206|207| 21|210|211|215|218| 22|222|224| 23|230|231|233|238|240|243|252|255|256|259|261|263|265|266|267|268|269| 27|271|273| 28|285|286|289| 29|293|296|297|298|306|308|313|315|316|319|320|321|329|332|339|341|344|346|347|350|351|354|355|356|357|361|363|366|371|372|375|378|380|382|383|386|392|395| 4|404|407|409|415|416|417|423|427| 43|432|435|437|438|439|444|445|446| 45|454|455|459| 46| 47|470|472| 48|502|505|510|511|520|521|524|529| 53|533|539|542|543|548| 55|551|552|558| 56|562|565|566|567|573|576|577| 58|583|584|585|586|589|591|592|599|605|608|609|611|617|619|629|630|631|632|633|634|637|638|639|647|648|649|651|652|654|656|657| 66|661|663|664|667|669| 67|673|675|677|678| 68|681|685|687|688|692|694|699|704|706|710|713|715|716|723|729|730|731|732|734| 74|741|749|751|752|756|757|762|763|764|776|777|778|779|780| 79|792|793|794|799| 8|801|804| 81|810|818|821|823|825|828|829|830|834| 84|846|853|855|856|857|859| 86|862|870|872|879| 88|882|886| 89|892|894|895|897|901|904|908|909|915|919|920|921|922|926| 93|931|935|938|940|942|944|948| 95|958|963|966|972|975|976|978| 98|980|983|984|985|986|988|989|990|991|994|999|
+---------+-------+------------+-------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 

In [22]:
%sql
select sum(Count_of_Untargeted_Genes) as Count_of_Untargeted_Genes ,sum(Count_of_Targeted_Genes) as Count_of_Targeted_Genes from
(
select count(geneid) as Count_of_Untargeted_Genes, 0 as Count_of_Targeted_Genes from genes where target = -1
union all
select 0 as Count_of_Untargeted_Genes, count(geneid) as Count_of_Targeted_Genes  from genes where target != -1
)A

Count_of_Untargeted_Genes,Count_of_Targeted_Genes
504,496


Count of targeted and untargeted Genes. The data contains several gene genomics which affect other gene sets.

In [24]:
%sql
-- Age Distribution
select age,count(patientid) as Patients_Age_Count from patients group by age order by age

age,Patients_Age_Count
15,7
16,9
17,8
18,6
19,16
20,13
21,14
22,16
23,16
24,22


Age distribution among the patients to understand demography

In [26]:
%sql
-- Gender Distribution
select gender,count(gender) as Gender_Count from patients group by gender

gender,Gender_Count
0,514
1,486


Distribution of Patients among Gender. According to graph , Gender Distribution is unbiased.

In [28]:
%sql

select  avg(drugResponse) as DrugResponse,case when age between 15 and 30 then '15-30' 
                           when age between 31 and 45 then '31-45' 
                           when age between 46 and 60 then '46-60' 
                           when age between 61 and 75 then '61-75'
                           when age between 76 and 95 then '76-95' end as Ages,
                           case when gender=0 then 'Male' else 'Female' end as Gender
          from patients  group by Ages,Gender order by Ages

DrugResponse,Ages,Gender
47.32608695652175,15-30,Female
57.12630000000001,15-30,Male
47.06239130434783,31-45,Male
47.60237113402063,31-45,Female
42.72232558139536,46-60,Male
51.38404761904762,46-60,Female
51.08663366336634,61-75,Male
51.69951923076923,61-75,Female
51.33925925925928,76-95,Male
50.05935779816512,76-95,Female


From the above graph, we can see the drug response based on the age and gender of the patients.

In [30]:
%sql
-- Count of Patients per disease
select disease as disease_id,count(patientid) as Disease_Count from patients group by disease order by Disease_Count desc

disease_id,Disease_Count
1,62
2,57
13,55
11,54
3,53
14,51
20,51
16,49
18,49
19,48


From the above graph, We see count of patients per disease.

In [32]:
g4.createOrReplaceTempView("g4")


In [33]:
# Create a 70-30 train test split

train_data,test_data=g4.randomSplit([0.7,0.3])

# Linear Regression

In [35]:
# Import the required libraries

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline

In [36]:
g4.columns[4:]

Out[22]: ['100',
 '106',
 '11',
 '116',
 '118',
 '120',
 '121',
 '123',
 '126',
 '136',
 '14',
 '140',
 '141',
 '144',
 '145',
 '149',
 '151',
 '152',
 '154',
 '16',
 '161',
 '163',
 '164',
 '169',
 '172',
 '174',
 '175',
 '182',
 '190',
 '194',
 '197',
 '198',
 '199',
 '201',
 '206',
 '207',
 '21',
 '210',
 '211',
 '215',
 '218',
 '22',
 '222',
 '224',
 '23',
 '230',
 '231',
 '233',
 '238',
 '240',
 '243',
 '252',
 '255',
 '256',
 '259',
 '261',
 '263',
 '265',
 '266',
 '267',
 '268',
 '269',
 '27',
 '271',
 '273',
 '28',
 '285',
 '286',
 '289',
 '29',
 '293',
 '296',
 '297',
 '298',
 '306',
 '308',
 '313',
 '315',
 '316',
 '319',
 '320',
 '321',
 '329',
 '332',
 '339',
 '341',
 '344',
 '346',
 '347',
 '350',
 '351',
 '354',
 '355',
 '356',
 '357',
 '361',
 '363',
 '366',
 '371',
 '372',
 '375',
 '378',
 '380',
 '382',
 '383',
 '386',
 '392',
 '395',
 '4',
 '404',
 '407',
 '409',
 '415',
 '416',
 '417',
 '423',
 '427',
 '43',
 '432',
 '435',
 '437',
 '438',
 '439',
 '444',
 '445',
 '446',
 '45',
 '454',
 '455',
 '459',
 '46',
 '47',
 '470',
 '472',
 '48',
 '502',
 '505',
 '510',
 '511',
 '520',
 '521',
 '524',
 '529',
 '53',
 '533',
 '539',
 '542',
 '543',
 '548',
 '55',
 '551',
 '552',
 '558',
 '56',
 '562',
 '565',
 '566',
 '567',
 '573',
 '576',
 '577',
 '58',
 '583',
 '584',
 '585',
 '586',
 '589',
 '591',
 '592',
 '599',
 '605',
 '608',
 '609',
 '611',
 '617',
 '619',
 '629',
 '630',
 '631',
 '632',
 '633',
 '634',
 '637',
 '638',
 '639',
 '647',
 '648',
 '649',
 '651',
 '652',
 '654',
 '656',
 '657',
 '66',
 '661',
 '663',
 '664',
 '667',
 '669',
 '67',
 '673',
 '675',
 '677',
 '678',
 '68',
 '681',
 '685',
 '687',
 '688',
 '692',
 '694',
 '699',
 '704',
 '706',
 '710',
 '713',
 '715',
 '716',
 '723',
 '729',
 '730',
 '731',
 '732',
 '734',
 '74',
 '741',
 '749',
 '751',
 '752',
 '756',
 '757',
 '762',
 '763',
 '764',
 '776',
 '777',
 '778',
 '779',
 '780',
 '79',
 '792',
 '793',
 '794',
 '799',
 '8',
 '801',
 '804',
 '81',
 '810',
 '818',
 '821',
 '823',
 '825',
 '828',
 '829',
 '830',
 '834',
 '84',
 '846',
 '853',
 '855',
 '856',
 '857',
 '859',
 '86',
 '862',
 '870',
 '872',
 '879',
 '88',
 '882',
 '886',
 '89',
 '892',
 '894',
 '895',
 '897',
 '901',
 '904',
 '908',
 '909',
 '915',
 '919',
 '920',
 '921',
 '922',
 '926',
 '93',
 '931',
 '935',
 '938',
 '940',
 '942',
 '944',
 '948',
 '95',
 '958',
 '963',
 '966',
 '972',
 '975',
 '976',
 '978',
 '98',
 '980',
 '983',
 '984',
 '985',
 '986',
 '988',
 '989',
 '990',
 '991',
 '994',
 '999']

In [37]:
# Vector assembler is used to create a vector of input features

assembler = VectorAssembler(inputCols=g4.columns[4:],
                            outputCol="features")

In [38]:
input_list  = g4.columns
input_list.append(assembler)

In [39]:
input_list

Out[25]: ['patientid',
 'disease',
 'drugResponse',
 'geo_patientid',
 '100',
 '106',
 '11',
 '116',
 '118',
 '120',
 '121',
 '123',
 '126',
 '136',
 '14',
 '140',
 '141',
 '144',
 '145',
 '149',
 '151',
 '152',
 '154',
 '16',
 '161',
 '163',
 '164',
 '169',
 '172',
 '174',
 '175',
 '182',
 '190',
 '194',
 '197',
 '198',
 '199',
 '201',
 '206',
 '207',
 '21',
 '210',
 '211',
 '215',
 '218',
 '22',
 '222',
 '224',
 '23',
 '230',
 '231',
 '233',
 '238',
 '240',
 '243',
 '252',
 '255',
 '256',
 '259',
 '261',
 '263',
 '265',
 '266',
 '267',
 '268',
 '269',
 '27',
 '271',
 '273',
 '28',
 '285',
 '286',
 '289',
 '29',
 '293',
 '296',
 '297',
 '298',
 '306',
 '308',
 '313',
 '315',
 '316',
 '319',
 '320',
 '321',
 '329',
 '332',
 '339',
 '341',
 '344',
 '346',
 '347',
 '350',
 '351',
 '354',
 '355',
 '356',
 '357',
 '361',
 '363',
 '366',
 '371',
 '372',
 '375',
 '378',
 '380',
 '382',
 '383',
 '386',
 '392',
 '395',
 '4',
 '404',
 '407',
 '409',
 '415',
 '416',
 '417',
 '423',
 '427',
 '43',
 '432',
 '435',
 '437',
 '438',
 '439',
 '444',
 '445',
 '446',
 '45',
 '454',
 '455',
 '459',
 '46',
 '47',
 '470',
 '472',
 '48',
 '502',
 '505',
 '510',
 '511',
 '520',
 '521',
 '524',
 '529',
 '53',
 '533',
 '539',
 '542',
 '543',
 '548',
 '55',
 '551',
 '552',
 '558',
 '56',
 '562',
 '565',
 '566',
 '567',
 '573',
 '576',
 '577',
 '58',
 '583',
 '584',
 '585',
 '586',
 '589',
 '591',
 '592',
 '599',
 '605',
 '608',
 '609',
 '611',
 '617',
 '619',
 '629',
 '630',
 '631',
 '632',
 '633',
 '634',
 '637',
 '638',
 '639',
 '647',
 '648',
 '649',
 '651',
 '652',
 '654',
 '656',
 '657',
 '66',
 '661',
 '663',
 '664',
 '667',
 '669',
 '67',
 '673',
 '675',
 '677',
 '678',
 '68',
 '681',
 '685',
 '687',
 '688',
 '692',
 '694',
 '699',
 '704',
 '706',
 '710',
 '713',
 '715',
 '716',
 '723',
 '729',
 '730',
 '731',
 '732',
 '734',
 '74',
 '741',
 '749',
 '751',
 '752',
 '756',
 '757',
 '762',
 '763',
 '764',
 '776',
 '777',
 '778',
 '779',
 '780',
 '79',
 '792',
 '793',
 '794',
 '799',
 '8',
 '801',
 '804',
 '81',
 '810',
 '818',
 '821',
 '823',
 '825',
 '828',
 '829',
 '830',
 '834',
 '84',
 '846',
 '853',
 '855',
 '856',
 '857',
 '859',
 '86',
 '862',
 '870',
 '872',
 '879',
 '88',
 '882',
 '886',
 '89',
 '892',
 '894',
 '895',
 '897',
 '901',
 '904',
 '908',
 '909',
 '915',
 '919',
 '920',
 '921',
 '922',
 '926',
 '93',
 '931',
 '935',
 '938',
 '940',
 '942',
 '944',
 '948',
 '95',
 '958',
 '963',
 '966',
 '972',
 '975',
 '976',
 '978',
 '98',
 '980',
 '983',
 '984',
 '985',
 '986',
 '988',
 '989',
 '990',
 '991',
 '994',
 '999',
 VectorAssembler_54209a9e6724]

In [40]:
# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data

pipe = Pipeline(stages=[assembler])

In [41]:
fitted_pipe=pipe.fit(train_data)

In [42]:
train_data=fitted_pipe.transform(train_data)

In [43]:
# Create an object for the Linear Regression model

lr_model = LinearRegression(labelCol='drugResponse')

In [44]:
# Fit the model on the train data

fit_model = lr_model.fit(train_data.select(['features','drugResponse']))

In [45]:
# Transform the test data using the model to predict the duration

test_data=fitted_pipe.transform(test_data)

In [46]:
train_data.show()

+---------+-------+------------+-------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------------------+
patientid|disease|drugResponse|geo_patientid| 100| 106| 11| 116| 118| 120| 121| 123| 126| 136| 14| 140| 141| 144| 145| 149| 151| 152| 154| 16| 161| 163| 164| 169| 172| 174| 175| 182| 190| 194| 197| 198| 199| 201| 206| 207| 21| 210| 211| 215| 218| 22| 222| 224| 23| 230| 231| 233| 238| 240| 243| 252| 255| 256| 259| 261| 263| 265| 266| 267| 268| 269| 27| 271| 273| 28| 285| 286| 289| 29| 293| 296| 297| 298| 306| 308| 313| 315| 316| 319| 320| 321| 329| 332| 339| 341| 

In [47]:
# Store the results in a dataframe

results = fit_model.transform(test_data)

In [48]:
results.select(['patientid','disease','drugResponse','prediction']).show()

+---------+-------+------------+------------------+
patientid|disease|drugResponse| prediction|
+---------+-------+------------+------------------+
 26| 7| 54.98| 79.68655385391848|
 191| 11| 22.58|112.15836600189432|
 541| 1| 41.02| 42.79235973694893|
 558| 13| 11.69| 29.0556830805206|
 222| 9| 96.71| 66.79615408481547|
 243| 6| 5.63| 77.71279211786434|
 367| 19| 3.97| 29.82663115878581|
 19| 1| 23.74|41.530662202725466|
 54| 17| 71.91| 38.53878098520203|
 0| 8| 58.14|52.369510466941314|
 348| 2| 0.17| 24.65246880462764|
 847| 12| 80.9| 60.7319929952032|
 736| 18| 41.48| 38.06061838527138|
 857| 20| 30.26| 50.28489612802704|
 22| 20| 87.23|57.440657148150116|
 196| 6| 6.24| 31.02047422982082|
 932| 20| 52.07|17.901968158153732|
 77| 0| 85.61| 75.44119392430237|
 677| 19| 58.94| 61.55786952043777|
 34| 19| 65.57| 72.22677206677747|
+---------+-------+------------+------------------+
only showing top 20 rows

##### Evaluating the model

In [51]:
test_results = fit_model.evaluate(test_data)

In [52]:
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
-24.706553853918486|
 -89.57836600189432|
-1.7723597369489283|
-17.365683080520597|
 29.913845915184524|
 -72.08279211786434|
 -25.85663115878581|
-17.790662202725468|
 33.371219014797965|
 5.7704895330586865|
 -24.48246880462764|
 20.168007004796806|
 3.4193816147286142|
 -20.02489612802704|
 29.789342851849888|
 -24.78047422982082|
 34.16803184184627|
 10.168806075697631|
-2.6178695204377718|
 -6.656772066777478|
+-------------------+
only showing top 20 rows

In [53]:
test_results.rootMeanSquaredError

Out[54]: 26.104369858541457

In [54]:
test_results.r2

Out[55]: 0.17141760889350122

In [56]:
# Create a 70-30 train test split

train_data,test_data=g4.randomSplit([0.7,0.3])

# Decision Tree Regressor

In [58]:
# Import the required libraries

from pyspark.ml.regression import DecisionTreeRegressor

In [59]:
fitted_pipe=pipe.fit(train_data)

In [60]:
train_data=fitted_pipe.transform(train_data)

In [63]:
# Create an object for the DecisionTree Regression model

dt_model = DecisionTreeRegressor(labelCol='drugResponse')

In [64]:
# Transform the test data using the model to predict the duration

test_data=fitted_pipe.transform(test_data)

In [66]:
# Fit the model on the train data

fit_model_dt = dt_model.fit(train_data.select(['features','drugResponse']))

In [67]:
# Store the results in a dataframe

results_dt = fit_model_dt.transform(test_data)

In [68]:
results_dt.select(['drugResponse','prediction']).show()

+------------+------------------+
drugResponse| prediction|
+------------+------------------+
 54.98| 60.6268115942029|
 72.61|33.404430379746834|
 11.69| 39.502|
 59.46|46.101755725190834|
 92.48|46.101755725190834|
 5.63| 61.39875000000001|
 70.11| 67.79854545454546|
 71.91|62.913999999999994|
 65.43| 36.85800000000004|
 58.14|46.101755725190834|
 0.17|62.913999999999994|
 80.9| 67.79854545454546|
 83.95|62.913999999999994|
 63.36| 58.69870967741935|
 75.13|46.101755725190834|
 83.75| 31.61615384615385|
 73.89|33.404430379746834|
 30.8|62.913999999999994|
 30.26|46.101755725190834|
 87.23| 67.79854545454546|
+------------+------------------+
only showing top 20 rows

##### Evaluating the model

In [70]:
# Select (prediction, true label) and compute test error
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="drugResponse", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(results_dt)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 33.3742

In [71]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="drugResponse", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(results_dt)
print("r2 on test data = %g" % r2)

r2 on test data = 0.140698

#RandomForest Regressor

In [73]:
# Create a 70-30 train test split

train_data,test_data=g4.randomSplit([0.7,0.3])

In [74]:
# Import the required libraries

from pyspark.ml.regression import RandomForestRegressor

In [75]:
fitted_pipe=pipe.fit(train_data)

In [76]:
train_data=fitted_pipe.transform(train_data)

In [77]:
# Create an object for the RandomForest Regression model

rf_model = RandomForestRegressor(labelCol='drugResponse')

In [78]:
# Fit the model on the train data

fit_model_rf = rf_model.fit(train_data.select(['features','drugResponse']))

In [79]:
# Transform the test data using the model to predict the duration

test_data=fitted_pipe.transform(test_data)

In [80]:
# Store the results in a dataframe

results_rf = fit_model_rf.transform(test_data)

In [81]:
results_dt.select(['drugResponse','prediction']).show()

+------------+------------------+
drugResponse| prediction|
+------------+------------------+
 72.61|33.404430379746834|
 92.48|46.101755725190834|
 79.01| 67.79854545454546|
 5.63| 61.39875000000001|
 85.65| 60.6268115942029|
 41.48|30.223125000000003|
 89.14| 67.79854545454546|
 87.47| 58.69870967741935|
 51.27|28.534000000000002|
 73.89|33.404430379746834|
 38.5| 58.69870967741935|
 30.8|62.913999999999994|
 13.98|46.101755725190834|
 51.65|35.858888888888885|
 27.37|33.404430379746834|
 96.44| 36.85800000000004|
 58.94|62.913999999999994|
 92.8| 80.62|
 67.3|46.101755725190834|
 64.81| 67.79854545454546|
+------------+------------------+
only showing top 20 rows

#### Evaluating the Random Forest Regressor model

In [83]:
# Select (prediction, true label) and compute test error
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="drugResponse", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(results_rf)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 30.4968

In [84]:
# Select (prediction, true label) and compute test error
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="drugResponse", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(results_rf)
print("r2 on test data = %g" % r2)

r2 on test data = 0.268879

####  Inferences
1) As we can see from the above results of both models , the predicted response rate for few patient ids seems to be quite close to the actual drug response.
For example, for patient id 474 the actual drug response rate and the predicted drug response rate are quite close. 

Thus, if we want to measure the drug response of a new patient whose Genes Expression are similar to that of patient id 474, our model will be fairly competent to suggest the drugResponse to that patient.

2) RandomForestRegressor is working the best for our data. The R-Square value is 26% which is the best among the 3 models.

2) But its also evident that the models are not very accurate. 
This is because we are working on a dataset 0f 1000 Rows to 1000 columns due to infrastructure constraints. In the Real World Genomics Benchmarking , this analysis is performed on upto 10^5 to 10^10 gene-sets per sample involving live stream of gene samples which gets updated periodically.

##### Find correlations between expression values of all pairs of genes to find genes which have similar expression patterns and genes which have opposing expression patterns.

In [88]:
from pyspark.ml.stat import Correlation
#g5 = g4.filter('disease == 18')
df_pre_corr = g5.select(g5.columns[4:])
df_for_corr = assembler.transform(df_pre_corr).select("features")

In [89]:
# get correlation matrix
matrix = Correlation.corr(df_for_corr, "features").collect()[0][0]

In [90]:
corrmatrix = matrix.toArray().tolist()
df_corr_matrix = spark.createDataFrame(corrmatrix,g5.columns[4:10].sort())
# print(corrmatrix)
display(df_corr_matrix)

_1,_2,_3,_4,_5,_6,_7,_8,_9,_10,_11,_12,_13,_14,_15,_16,_17,_18,_19,_20,_21,_22,_23,_24,_25,_26,_27,_28,_29,_30,_31,_32,_33,_34,_35,_36,_37,_38,_39,_40,_41,_42,_43,_44,_45,_46,_47,_48,_49,_50,_51,_52,_53,_54,_55,_56,_57,_58,_59,_60,_61,_62,_63,_64,_65,_66,_67,_68,_69,_70,_71,_72,_73,_74,_75,_76,_77,_78,_79,_80,_81,_82,_83,_84,_85,_86,_87,_88,_89,_90,_91,_92,_93,_94,_95,_96,_97,_98,_99,_100,_101,_102,_103,_104,_105,_106,_107,_108,_109,_110,_111,_112,_113,_114,_115,_116,_117,_118,_119,_120,_121,_122,_123,_124,_125,_126,_127,_128,_129,_130,_131,_132,_133,_134,_135,_136,_137,_138,_139,_140,_141,_142,_143,_144,_145,_146,_147,_148,_149,_150,_151,_152,_153,_154,_155,_156,_157,_158,_159,_160,_161,_162,_163,_164,_165,_166,_167,_168,_169,_170,_171,_172,_173,_174,_175,_176,_177,_178,_179,_180,_181,_182,_183,_184,_185,_186,_187,_188,_189,_190,_191,_192,_193,_194,_195,_196,_197,_198,_199,_200,_201,_202,_203,_204,_205,_206,_207,_208,_209,_210,_211,_212,_213,_214,_215,_216,_217,_218,_219,_220,_221,_222,_223,_224,_225,_226,_227,_228,_229,_230,_231,_232,_233,_234,_235,_236,_237,_238,_239,_240,_241,_242,_243,_244,_245,_246,_247,_248,_249,_250,_251,_252,_253,_254,_255,_256,_257,_258,_259,_260,_261,_262,_263,_264,_265,_266,_267,_268,_269,_270,_271,_272,_273,_274,_275,_276,_277,_278,_279,_280,_281,_282,_283,_284,_285,_286,_287,_288,_289,_290,_291,_292,_293,_294,_295,_296,_297,_298,_299,_300,_301,_302,_303,_304,_305,_306,_307,_308,_309,_310,_311,_312,_313,_314,_315
1.0,-0.0786875698687383,-0.12487239731176493,0.02557440684318347,-0.12807959871483526,0.2155377465870811,0.11933553133668098,-0.04929083871418642,-0.118242323828244,0.022328271902318418,0.13034396783981902,-0.05029319832329885,0.1707180188634201,0.053885785401559874,0.009736763289003778,-0.057313292951079155,-0.21822031688256377,-0.07476427904767383,0.06967857068507495,0.08878308189543087,-0.1103611101366197,0.17585383279608674,-0.10261067008341018,-0.00655460893967346,-0.08161598650498672,-0.2947041606259353,0.2092109205369039,-0.22422071314757297,0.3326808519171335,0.045863424757323766,0.07319051656543671,0.07346988795822006,0.27499976850422847,-0.04831624001076915,0.07825299522745478,-0.006997771620029689,0.15182559063176287,-0.3114624893281102,-0.1666706942095912,0.06448406342367896,0.13007587600645934,0.013477151723922543,-0.17782925757281995,0.07717360703277613,-0.03612723550990619,-0.06553023892210004,0.10681385286646167,0.21882869286436654,-0.04739372797581997,-0.1514638258622984,0.058920659824444485,-0.10416558527132572,-0.18320234675193484,-0.07788863164542222,-0.15481539488347273,0.03548755571121599,0.11211312185787498,-0.11450241570683373,-0.22322485092199335,-0.16816554257322247,-0.1718714139548953,0.23074562006157123,-0.027762052787335234,0.1064458054170518,-0.08022309388715389,0.08758428248975217,0.1327519580068598,0.10743525115268286,0.10542400674929818,0.19537566500800604,-0.002028293756559919,-0.11235676943304074,0.0460148365543176,-0.05505914070313945,-0.20436845569415674,-0.07979531884232893,0.07012676207679074,-0.17642040511913493,-0.18481468278421248,0.020130340779087715,0.006791665835195889,-0.011000248157509239,0.06005002449617986,-0.02241688784926343,0.08793456125259685,-0.011185373960139553,0.13511971462320635,-0.100835503413501,-0.02392244505839554,0.02673444085107506,0.14029752992551184,-0.1256501832057446,0.25213710924158594,0.07077917730222356,0.16764383050346007,-0.1264461844008868,-0.03764928205487587,-0.17227817761536238,0.036508098415803936,0.18181059116416806,0.03901723008182293,0.18445250421062115,-0.08879364134225637,-0.25107321541082356,0.08066189171409897,-0.18298059250464732,-0.04310365149663448,0.08926390382775765,-0.07565963519882105,-0.037950174783327247,0.1129823692192135,0.10192220813071363,0.12314656461978495,-0.05866932344406473,0.12956942496133111,0.3373970252610463,0.0029832152949870127,0.1644996946400562,-0.12349663318557969,0.2993231647102773,-0.015257996974564366,0.052012753949872176,0.2582402567327183,-0.0418376446951313,0.0485547660134

In [91]:
import matplotlib.pyplot as plt

def plot_corr_matrix(correlations,attr,fig_no):
    fig=plt.figure(fig_no)
    ax=fig.add_subplot(111)
    ax.set_title("Correlation Matrix for Specified Attributes")
    ax.set_xticklabels(['']+attr)
    ax.set_yticklabels(['']+attr)
    cax=ax.matshow(correlations,vmax=1,vmin=-1)
    fig.colorbar(cax)
    plt.show()
    
    
    
plot_corr_matrix(corrmatrix, g5.columns[4:10] , 234)